In [15]:
!pip install gspread oauth2client transformers torch

In [14]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# Autentikasi Google Sheets API
auth.authenticate_user()
creds, _ = default()
gspread_client = gspread.authorize(creds)

# Memuat Google Sheet
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1fFm00AK3cZeYu1kGDgr7wRHuCVOYQ46u3VBH2D0o23k/edit?usp=drive_link'
sheet = gspread_client.open_by_url(spreadsheet_url).sheet1

# Konversi data menjadi DataFrame pandas
data = sheet.get_all_values()
headers = data.pop(0)
comments_df = pd.DataFrame(data, columns=headers)

# Menampilkan beberapa data
print(comments_df.head(10))


                                            Comments
0  oppo indonesia : open recruitment for cmo oppo...
1                                                   
2                         kelihatan kualitasnya wkwk
3                               <a href=" <a href=" 
4                                             sampah
5  oppo clearly needs to sign another indonesian ...
6  wohahahahaha brand hp paling bnyak rusak di ko...
7   ga pernah tertarik ama smartphone dari brand ini
8  this is an insult, they think that we are stup...
9  yaelah kirain terima kritikan malah mencari pe...


In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("indolem/indobert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
from transformers import pipeline

# Menggunakan model lain yang tersedia secara publik
sentiment_pipeline = pipeline("sentiment-analysis", model="indolem/indobert-base-uncased")

# Definisikan fungsi untuk mendapatkan sentimen
def get_sentiment(comment):
    result = sentiment_pipeline(comment)[0]
    return "positif" if result['label'] == 'positive' else "negatif"   # 1 untuk positif, 0 untuk negatif

# Menerapkan analisis sentimen ke kolom 'Comments'
comments_df['Sentiment'] = comments_df['Comments'].apply(get_sentiment)

# Menampilkan beberapa data setelah penambahan kolom 'Sentiment'
print(comments_df.head(10))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                                            Comments Sentiment
0  oppo indonesia : open recruitment for cmo oppo...   negatif
1                                                      negatif
2                         kelihatan kualitasnya wkwk   negatif
3                               <a href=" <a href="    negatif
4                                             sampah   negatif
5  oppo clearly needs to sign another indonesian ...   negatif
6  wohahahahaha brand hp paling bnyak rusak di ko...   negatif
7   ga pernah tertarik ama smartphone dari brand ini   negatif
8  this is an insult, they think that we are stup...   negatif
9  yaelah kirain terima kritikan malah mencari pe...   negatif


In [26]:
# Menyimpan DataFrame yang berisi hasil analisis sentimen
comments_df.to_csv('/content/drive/My Drive/KULIAH/SEM 5/APLIKASI WEB/dataset/comments_sentiment.csv', index=False)
